In [2]:
import pyedflib
import numpy as np

def read_edf_signals(edf_file):
    f = pyedflib.EdfReader(edf_file)
    num_signals = f.signals_in_file


    signals = []
    for i in range(num_signals):
        signal = f.readSignal(i)
        signals.append(signal)

    labels = f.getSignalLabels()
    sampling_rates = f.getSampleFrequencies()

    f.close()
    
    return signals, labels, sampling_rates

# edf_file_path = "./dataset/files/ucddb002.rec"
# signals, labels, sampling_rates = read_edf_signals(edf_file_path)

# print("Signal labels:", labels)
# print("Sampling rates:", sampling_rates)

# print("First 10 samples of the first signal:", signals[0][:10]) 
# Imp signals -> 'Flow', 'ribcage'

x_list = []
y_list = []

for i in range(2, 29):
    try:
        if i<10:
            sleep_stages = np.loadtxt(f"dataset/files/ucddb00{i}_stage.txt", dtype=int)
            edf_file_path = f"./dataset/files/ucddb00{i}.rec"
            signals, labels, sampling_rates = read_edf_signals(edf_file_path)
        else:
            sleep_stages = np.loadtxt(f"dataset/files/ucddb0{i}_stage.txt", dtype=int)
            edf_file_path = f"./dataset/files/ucddb0{i}.rec"
            signals, labels, sampling_rates = read_edf_signals(edf_file_path)

        x_list.append([signals[8], signals[10]])
        y_list.append(sleep_stages)
    except:
        print(f"Error in file {i}")
        continue

    print()
    print()
    print(f"File {i} has {len(signals)} signals", \
          "\nsamples [nasal, ribcage]", f"[{len(signals[8])}, {len(signals[10])}]", \
          "\nsampling rate [nasal, ribcage]", f"[{sampling_rates[8]}, {sampling_rates[10]}]",\
          "\nno of sleep stages", len(signals[8])/(30*sampling_rates[8]), len(signals[10])/(30*sampling_rates[10]),  \
          "\nSleep stages: ", len(sleep_stages))    




File 2 has 14 signals 
samples [nasal, ribcage] [179760, 179760] 
sampling rate [nasal, ribcage] [8.0, 8.0] 
no of sleep stages 749.0 749.0 
Sleep stages:  748


File 3 has 14 signals 
samples [nasal, ribcage] [211824, 211824] 
sampling rate [nasal, ribcage] [8.0, 8.0] 
no of sleep stages 882.6 882.6 
Sleep stages:  882
Error in file 4


File 5 has 14 signals 
samples [nasal, ribcage] [198384, 198384] 
sampling rate [nasal, ribcage] [8.0, 8.0] 
no of sleep stages 826.6 826.6 
Sleep stages:  826


File 6 has 14 signals 
samples [nasal, ribcage] [194136, 194136] 
sampling rate [nasal, ribcage] [8.0, 8.0] 
no of sleep stages 808.9 808.9 
Sleep stages:  808


File 7 has 14 signals 
samples [nasal, ribcage] [195240, 195240] 
sampling rate [nasal, ribcage] [8.0, 8.0] 
no of sleep stages 813.5 813.5 
Sleep stages:  813


File 8 has 14 signals 
samples [nasal, ribcage] [184328, 184328] 
sampling rate [nasal, ribcage] [8.0, 8.0] 
no of sleep stages 768.0333333333333 768.0333333333333 
Sleep s

In [3]:
reshaped_x_list = []
for i in range(len(x_list)):
    array1 = np.array(x_list[i][0])
    array2 = np.array(x_list[i][1])

    truncated_array1 = array1[:len(y_list[i])*240]
    truncated_array2 = array2[:len(y_list[i])*240]

    reshaped1 = truncated_array1.reshape(-1, 240) 
    reshaped2 = truncated_array2.reshape(-1, 240) 

    final_array = np.stack([reshaped1, reshaped2], axis=1)
    reshaped_x_list.append(final_array)

    print(array1.shape, array2.shape, y_list[i].shape, final_array.shape)

print(len(reshaped_x_list))

(179760,) (179760,) (748,) (748, 2, 240)
(211824,) (211824,) (882,) (882, 2, 240)
(198384,) (198384,) (826,) (826, 2, 240)
(194136,) (194136,) (808,) (808, 2, 240)
(195240,) (195240,) (813,) (813, 2, 240)
(184328,) (184328,) (768,) (768, 2, 240)
(222072,) (222072,) (925,) (925, 2, 240)
(217688,) (217688,) (907,) (907, 2, 240)
(216240,) (216240,) (900,) (900, 2, 240)
(207528,) (207528,) (864,) (864, 2, 240)
(194664,) (194664,) (811,) (811, 2, 240)
(185912,) (185912,) (774,) (774, 2, 240)
(219904,) (219904,) (916,) (916, 2, 240)
(189472,) (189472,) (789,) (789, 2, 240)
(197480,) (197480,) (822,) (822, 2, 240)
(204584,) (204584,) (852,) (852, 2, 240)
(180688,) (180688,) (752,) (752, 2, 240)
(219272,) (219272,) (913,) (913, 2, 240)
(189120,) (189120,) (787,) (787, 2, 240)
(206800,) (206800,) (861,) (861, 2, 240)
(218000,) (218000,) (908,) (908, 2, 240)
(170800,) (170800,) (711,) (711, 2, 240)
(201280,) (201280,) (838,) (838, 2, 240)
(214328,) (214328,) (893,) (893, 2, 240)
(173280,) (17328

In [4]:
for i in range(len(y_list)):
    y_list[i] = y_list[i].reshape(-1, 1)

In [5]:
y_list[0].shape

(748, 1)

In [6]:
subject_wise_data = list(zip(reshaped_x_list, y_list))

binary_y_list = [] # each element is from a subject
for i in range(len(y_list)):
    binary_y_list.append((y_list[i] > 0).astype(int))

np.unique(binary_y_list[0], return_counts=True)

binary_subject_wise_data = list(zip(reshaped_x_list, binary_y_list))

In [7]:
binary_subject_wise_data[0][1].shape, binary_subject_wise_data[0][0].shape

((748, 1), (748, 2, 240))

In [8]:
import torch
from torch.utils.data import DataLoader, Dataset


### HYPERPARAMTERS

In [9]:
batch_size = 32

In [10]:
%pip install xgboost

Note: you may need to restart the kernel to use updated packages.


## XG boost

In [ ]:
train_data = [x for j, x in enumerate(binary_subject_wise_data) if j != i]

    # Concatenate features and labels for training
train_labels = np.hstack([x[1].ravel() for x in train_data])
train_labels.shape

(19175,)

In [ ]:
train_data = [x for j, x in enumerate(binary_subject_wise_data) if j != i]

# Concatenate features and labels for training
train_features = np.vstack([x[0] for x in train_data])
train_labels = np.vstack([x[1] for x in train_data])

In [ ]:
train_features.shape

(19175, 2, 240)

In [ ]:
import tsfel

cfg = tsfel.get_features_by_domain('temporal')  # Get all temporal features

features_list = []
count = 0
for sample in train_features:
    features = []
    for ch in range(train_features.shape[1]):  # Iterate over channels (2 in this case)
        channel_data = sample[ch]
        if np.nan in sample or np.inf in sample:
            count+=1
        else:
            X_ch = tsfel.time_series_features_extractor(cfg, sample, fs=100)
            X_ch = np.squeeze(X_ch)
            features.append(X_ch)
    features_list.append(features)

print("Final feature shape:", features_list)  # Should be (19175, feature_count * 2)


NameError: name 'train_features' is not defined

In [25]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


for i, data in enumerate(binary_subject_wise_data):
    print(f"Leave-One-Subject-Out CV - Subject: {i+1}\n-----------------------------------")
    
    # Split data: leave out the current subject for testing
    train_data = [x for j, x in enumerate(binary_subject_wise_data) if j != i]
    test_data = data

    # Concatenate features and labels for training
    train_features = np.vstack([x[0] for x in train_data])
    train_labels = np.vstack([x[1] for x in train_data])
    test_features = test_data[0]
    test_labels = test_data[1]

    X_train = train_features.reshape(-1, 2*240)
    y_train = train_labels
    X_test = test_features.reshape(-1, 2*240)
    y_test = test_labels



    # Train XGBoost classifier
    model = xgb.XGBClassifier(objective="binary:logistic", eval_metric="logloss", use_label_encoder=False)
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)

    tn, fp, fn, tp = confusion_matrix(test_labels, y_pred).ravel()

    # Compute metrics
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    precision = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall = tp / (tp + fn) if (tp + fn) != 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) != 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) != 0 else 0
    fpr = fp / (fp + tn) if (fp + tn) != 0 else 0
    fnr = fn / (tp + fn) if (tp + fn) != 0 else 0

    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall (Sensitivity): {recall:.4f}")
    print(f"Specificity: {specificity:.4f}")
    print(f"F1-score: {f1:.4f}")
    print(f"False Positive Rate (FPR): {fpr:.4f}")
    print(f"False Negative Rate (FNR): {fnr:.4f}\n")

    # Accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Test Accuracy: {accuracy:.4f}")

Leave-One-Subject-Out CV - Subject: 1
-----------------------------------


d:\clg_stuff\ML project\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [19:42:51] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.8489
Precision: 0.8768
Recall (Sensitivity): 0.9537
Specificity: 0.3058
F1-score: 0.9137
False Positive Rate (FPR): 0.6942
False Negative Rate (FNR): 0.0463

Test Accuracy: 0.8489
Leave-One-Subject-Out CV - Subject: 2
-----------------------------------


d:\clg_stuff\ML project\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [19:42:59] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.8696
Precision: 0.8875
Recall (Sensitivity): 0.9608
Specificity: 0.4821
F1-score: 0.9227
False Positive Rate (FPR): 0.5179
False Negative Rate (FNR): 0.0392

Test Accuracy: 0.8696
Leave-One-Subject-Out CV - Subject: 3
-----------------------------------


...

Test Accuracy: 0.7539
Precision: 0.8324
Recall (Sensitivity): 0.8522
Specificity: 0.4233
F1-score: 0.8422
False Positive Rate (FPR): 0.5767
False Negative Rate (FNR): 0.1478

Test Accuracy: 0.7539
Leave-One-Subject-Out CV - Subject: 23
-----------------------------------


d:\clg_stuff\ML project\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [19:45:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.8842
Precision: 0.8966
Recall (Sensitivity): 0.9793
Specificity: 0.2679
F1-score: 0.9361
False Positive Rate (FPR): 0.7321
False Negative Rate (FNR): 0.0207

Test Accuracy: 0.8842
Leave-One-Subject-Out CV - Subject: 24
-----------------------------------


d:\clg_stuff\ML project\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [19:45:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.8690
Precision: 0.8840
Recall (Sensitivity): 0.9752
Specificity: 0.2283
F1-score: 0.9274
False Positive Rate (FPR): 0.7717
False Negative Rate (FNR): 0.0248

Test Accuracy: 0.8690


In [12]:
import xgboost as xgb
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import numpy as np

def combine_epochs(signals, labels):
    """
    Given:
      signals: numpy array of shape (n_epochs, 2, 240)
      labels: numpy array of shape (n_epochs,) or (n_epochs, 1)
      
    Returns:
      combined_signals: numpy array of shape (n_epochs-2, 3*2*240)
      combined_labels: numpy array of shape (n_epochs-2,)
    
    For each epoch index i (starting from 2), we combine:
       - signals[i-2]  (previous-to-previous epoch)
       - signals[i-1]  (previous epoch)
       - signals[i]    (current epoch)
    and use the label from the current epoch.
    """
    new_signals = []
    new_labels = []
    # Ensure labels are 1D
    labels = labels.ravel()
    for i in range(2, signals.shape[0]):
        # Flatten each epoch (2,240) to 480 and concatenate three of them:
        combined = np.concatenate((
            signals[i-2].reshape(-1),
            signals[i-1].reshape(-1),
            signals[i].reshape(-1)
        ))
        new_signals.append(combined)
        new_labels.append(labels[i])
    return np.array(new_signals), np.array(new_labels)

quant_dict = {}

# LOSO-CV loop
for i, data in enumerate(binary_subject_wise_data):
    print(f"Leave-One-Subject-Out CV - Subject: {i+1}\n-----------------------------------")
    
    # Leave current subject for testing; all others for training.
    train_data = [x for j, x in enumerate(binary_subject_wise_data) if j != i]
    test_data = binary_subject_wise_data[i]

    # For training, combine epochs from each training subject:
    X_train_list, y_train_list = [], []
    for subject in train_data:
        signals = subject[0]   # shape: (n_epochs, 2, 240)
        labels = subject[1]    # shape: (n_epochs, 1) or (n_epochs,)
        X_subj, y_subj = combine_epochs(signals, labels)
        X_train_list.append(X_subj)
        y_train_list.append(y_subj)
    X_train = np.vstack(X_train_list)
    y_train = np.hstack(y_train_list)

    # For the test subject, combine epochs:
    signals_test = test_data[0]   # shape: (n_epochs, 2, 240)
    labels_test = test_data[1]
    X_test, y_test = combine_epochs(signals_test, labels_test)

    print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")
    
    # Train XGBoost classifier
    model = xgb.XGBClassifier(
        objective="binary:logistic", 
        eval_metric="logloss", 
        use_label_encoder=False,
        max_depth=3,
        learning_rate=0.05,
        n_estimators=300
    )
    model.fit(X_train, y_train)

    # Predictions and metrics
    y_pred = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    accuracy_val = (tp + tn) / (tp + tn + fp + fn)
    precision_val = tp / (tp + fp) if (tp + fp) != 0 else 0
    recall_val = tp / (tp + fn) if (tp + fn) != 0 else 0
    specificity_val = tn / (tn + fp) if (tn + fp) != 0 else 0
    f1_val = 2 * (precision_val * recall_val) / (precision_val + recall_val) if (precision_val + recall_val) != 0 else 0
    fpr_val = fp / (fp + tn) if (fp + tn) != 0 else 0
    fnr_val = fn / (tp + fn) if (tp + fn) != 0 else 0

    print(f"Test Accuracy: {accuracy_val:.4f}")
    print(f"Precision: {precision_val:.4f}")
    print(f"Recall (Sensitivity): {recall_val:.4f}")
    print(f"Specificity: {specificity_val:.4f}")
    print(f"F1-score: {f1_val:.4f}")
    print(f"False Positive Rate (FPR): {fpr_val:.4f}")
    print(f"False Negative Rate (FNR): {fnr_val:.4f}\n")
    print(f"Test Accuracy (sklearn): {accuracy_score(y_test, y_pred):.4f}\n")

    quant_dict[i] = {
        "accuracy": accuracy_val,
        "precision": precision_val,
        "recall": recall_val,
        "specificity": specificity_val,
        "f1": f1_val,
        "fpr": fpr_val,
        "fnr": fnr_val
    }


Leave-One-Subject-Out CV - Subject: 1
-----------------------------------
Train shape: (19993, 1440), Test shape: (746, 1440)


d:\clg_stuff\ML project\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [22:57:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.8619
Precision: 0.8711
Recall (Sensitivity): 0.9809
Specificity: 0.2353
F1-score: 0.9227
False Positive Rate (FPR): 0.7647
False Negative Rate (FNR): 0.0191

Test Accuracy (sklearn): 0.8619

Leave-One-Subject-Out CV - Subject: 2
-----------------------------------
Train shape: (19859, 1440), Test shape: (880, 1440)


d:\clg_stuff\ML project\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [22:57:54] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.8625
Precision: 0.8568
Recall (Sensitivity): 0.9972
Specificity: 0.2831
F1-score: 0.9217
False Positive Rate (FPR): 0.7169
False Negative Rate (FNR): 0.0028

Test Accuracy (sklearn): 0.8625

Leave-One-Subject-Out CV - Subject: 3
-----------------------------------
Train shape: (19915, 1440), Test shape: (824, 1440)


...

Test Accuracy: 0.8828
Precision: 0.8839
Recall (Sensitivity): 0.9959
Specificity: 0.1364
F1-score: 0.9365
False Positive Rate (FPR): 0.8636
False Negative Rate (FNR): 0.0041

Test Accuracy (sklearn): 0.8828

Leave-One-Subject-Out CV - Subject: 24
-----------------------------------
Train shape: (19848, 1440), Test shape: (891, 1440)


d:\clg_stuff\ML project\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [23:07:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.8788
Precision: 0.8834
Recall (Sensitivity): 0.9896
Specificity: 0.2000
F1-score: 0.9335
False Positive Rate (FPR): 0.8000
False Negative Rate (FNR): 0.0104

Test Accuracy (sklearn): 0.8788

Leave-One-Subject-Out CV - Subject: 25
-----------------------------------
Train shape: (20020, 1440), Test shape: (719, 1440)


d:\clg_stuff\ML project\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [23:07:30] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test Accuracy: 0.7121
Precision: 0.7072
Recall (Sensitivity): 0.9899
Specificity: 0.1062
F1-score: 0.8250
False Positive Rate (FPR): 0.8938
False Negative Rate (FNR): 0.0101

Test Accuracy (sklearn): 0.7121



In [13]:
acc = 0
spec = 0
for i in range(len(quant_dict)):
    acc += quant_dict[i]["accuracy"]
    spec += quant_dict[i]["specificity"]
spec/len(quant_dict), acc/len(quant_dict)

(0.25583358707558324, 0.7925138248841168)